In [1]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout


In [ ]:
#Reading structure and force field files
pdb = PDBFile('/home/arnau/Desktop/test.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

#Creating System
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME, nonbondedCutoff=1*nanometer, constraints=HBonds)
system.addForce(MonteCarloBarostat(1*bar, 300*kelvin))
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)

#Creating simulation context
print('Building system...')
platform = Platform.getPlatformByName('CUDA')
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

#Minimizing energy
print('Performing energy minimization...')
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000,
            step=True, potentialEnergy=True, kineticEnergy=True, totalEnergy=True, temperature=True, separator='\t'))

#Running simulation
print('Simulating...')
simulation.step(10000)
print('Simulation done')